<a href="https://colab.research.google.com/github/Domaakshithareddy/NeuroSpeech/blob/main/Major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

akshithawork_cloud_json_path = kagglehub.dataset_download('akshithawork/cloud-json')
akshithawork_librispeech_100_path = kagglehub.dataset_download('akshithawork/librispeech-100')
akshithawork_wav2vec2_large_robust_local_path = kagglehub.dataset_download('akshithawork/wav2vec2-large-robust-local')

print('Data source import complete.')


In [ ]:
# cell: detect_librispeech_and_preview.py
import os
from pathlib import Path

# candidate roots to search for LibriSpeech
candidates = [
    Path("/kaggle/input"),
    Path("/kaggle/working"),
    Path("/kaggle/working/data"),
    Path("/mnt/data"),
    Path(".")
]

found = None
for cand in candidates:
    if cand.exists():
        # search for a folder named 'LibriSpeech' or 'train-clean-100' directly
        libdir = cand / "LibriSpeech"
        if libdir.exists() and (libdir / "train-clean-100").exists():
            found = libdir
            break
        # maybe they uploaded as top-level train-clean-100
        if (cand / "train-clean-100").exists():
            found = cand
            break

if not found:
    # fallback: search recursively for a train-clean-100 folder
    for root, dirs, files in os.walk("/kaggle", topdown=True):
        if "train-clean-100" in dirs:
            found = Path(root)
            break
    if not found:
        for root, dirs, files in os.walk("/mnt", topdown=True):
            if "train-clean-100" in dirs:
                found = Path(root)
                break

if not found:
    print("Could not auto-detect LibriSpeech. Please paste the full path to the folder containing 'train-clean-100'.")
else:
    print("Detected dataset root:", found)
    t = (found / "train-clean-100")
    print("train-clean-100 exists:", t.exists())
    # list first-level children and show sample audio files for one speaker
    print("\nTop-level entries in dataset root:")
    for p in sorted(found.iterdir()):
        print(" -", p.name, "(dir)" if p.is_dir() else "(file)", )
    # show a few speaker folders
    if t.exists():
        speakers = sorted([d for d in t.iterdir() if d.is_dir()])[:8]
        print("\nSample speaker folders (first 8):", [s.name for s in speakers])
        # show one speaker/chapter contents
        if speakers:
            sp = speakers[0]
            chapters = sorted([d for d in sp.iterdir() if d.is_dir()])[:4]
            print(f"\nFor speaker {sp.name}, show up to 4 chapters:", [c.name for c in chapters])
            if chapters:
                ch = chapters[0]
                files = sorted(list(ch.glob("*.*")))[:10]
                print(f"\nExample files in {sp.name}/{ch.name}:")
                for f in files:
                    print("   ", f.name, "-", f.stat().st_size, "bytes")


Detected dataset root: /kaggle/input/librispeech-100/LibriSpeech
train-clean-100 exists: True

Top-level entries in dataset root:
 - BOOKS.TXT (file)
 - CHAPTERS.TXT (file)
 - LICENSE.TXT (file)
 - README.TXT (file)
 - SPEAKERS.TXT (file)
 - train-clean-100 (dir)

Sample speaker folders (first 8): ['103', '1034', '1040', '1069', '1081', '1088', '1098', '1116']

For speaker 103, show up to 4 chapters: ['1240', '1241']

Example files in 103/1240:
    103-1240-0000.flac - 255398 bytes
    103-1240-0001.flac - 301627 bytes
    103-1240-0002.flac - 270539 bytes
    103-1240-0003.flac - 277620 bytes
    103-1240-0004.flac - 239259 bytes
    103-1240-0005.flac - 295296 bytes
    103-1240-0006.flac - 183331 bytes
    103-1240-0007.flac - 281327 bytes
    103-1240-0008.flac - 287677 bytes
    103-1240-0009.flac - 196792 bytes


In [ ]:
# Build manifest directly from .trans.txt files (robust, no torchaudio)
from pathlib import Path
import csv
import sys

dataset_root = Path("/kaggle/input/librispeech-100/LibriSpeech")
split = "train-clean-100"
split_dir = dataset_root / split
if not split_dir.exists():
    raise RuntimeError(f"Expected {split_dir} to exist. Adjust dataset_root if needed.")

# 1) collect all .trans.txt files and build mapping utterance_id -> transcript
trans_map = {}
trans_files = list(split_dir.glob("**/*.trans.txt"))
print(f"Found {len(trans_files)} .trans.txt files (chapter-level transcript files).")

for tf in trans_files:
    try:
        with tf.open("r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                # format: "<utterance-id> <transcript...>"
                parts = line.split(" ", 1)
                if len(parts) == 2:
                    uttid, text = parts
                    trans_map[uttid] = text
    except Exception as e:
        print("Warning: failed reading", tf, ":", e)

print("Total utterances in transcripts mapping:", len(trans_map))

# 2) find all .flac files and match to trans_map
flac_files = list(split_dir.glob("**/*.flac"))
print("Found", len(flac_files), "flac files under", split_dir)

manifest_out = Path("manifests")
manifest_out.mkdir(exist_ok=True)
out_file = manifest_out / f"{split}.tsv"

write_count = 0
missing_count = 0
with out_file.open("w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["audio_path", "transcript"])
    for fl in sorted(flac_files):
        # utterance id is filename without extension (e.g., 103-1240-0000)
        uttid = fl.stem
        if uttid in trans_map:
            writer.writerow([str(fl), trans_map[uttid]])
            write_count += 1
        else:
            missing_count += 1
            # Optionally write with empty transcript or skip. We'll skip but log.
            # writer.writerow([str(fl), ""])
if missing_count:
    print(f"Warning: {missing_count} flac files had no matching transcript (they were skipped).")
print(f"Wrote {write_count} entries to {out_file.resolve()}")

# 3) print a few sample rows
print("\nSample manifest lines:")
with out_file.open("r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 5: break

Found 585 .trans.txt files (chapter-level transcript files).
Total utterances in transcripts mapping: 28539
Found 28539 flac files under /kaggle/input/librispeech-100/LibriSpeech/train-clean-100
Wrote 28539 entries to /kaggle/working/manifests/train-clean-100.tsv

Sample manifest lines:
audio_path	transcript
/kaggle/input/librispeech-100/LibriSpeech/train-clean-100/103/1240/103-1240-0000.flac	CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK
/kaggle/input/librispeech-100/LibriSpeech/train-clean-100/103/1240/103-1240-0001.flac	THAT HAD ITS SOURCE AWAY BACK IN THE WOODS OF THE OLD CUTHBERT PLACE IT WAS REPUTED TO BE AN INTRICATE HEADLONG BROOK IN ITS EARLIER COURSE THROUGH THOSE WOODS WITH DARK SECRETS OF POOL AND CASCADE BUT BY THE TIME IT REACHED LYNDE'S HOLLOW IT WAS A QUIET WELL CONDUCTED LITTLE STREAM
/kaggle/input/librispeech-100/Li

In [ ]:
# Cell 1: env & imports
import os
from pathlib import Path
import csv
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Processor
import nltk
nltk.download("cmudict", quiet=True)
from nltk.corpus import cmudict

# Paths (edit if your locations differ)
DATA_ROOT = Path("/kaggle/input/librispeech-100/LibriSpeech")   # where the audio folders are
MANIFEST_PATH = Path("/kaggle/working/manifests/train-clean-100.tsv")          # your uploaded manifest in this chat

print("DATA_ROOT exists:", DATA_ROOT.exists(), "  path:", DATA_ROOT)
print("MANIFEST_PATH exists:", MANIFEST_PATH.exists(), "  path:", MANIFEST_PATH)

# quick device print
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

DATA_ROOT exists: True   path: /kaggle/input/librispeech-100/LibriSpeech
MANIFEST_PATH exists: True   path: /kaggle/working/manifests/train-clean-100.tsv
Device: cpu


In [ ]:
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor

LOCAL = "/kaggle/input/wav2vec2-large-robust-local/wav2vec2-large-robust-local"  # or wherever you unzipped it
model = Wav2Vec2Model.from_pretrained(LOCAL, local_files_only=True)
fe = Wav2Vec2FeatureExtractor.from_pretrained(LOCAL, local_files_only=True)

print("Model loaded:", model.config.hidden_size)
print("Feature extractor sr:", fe.sampling_rate)

Model loaded: 1024
Feature extractor sr: 16000


In [ ]:
# Single cell: small end-to-end sanity check (one prompt)
import os
from pathlib import Path
import csv
import torchaudio
import torch
import torch.nn as nn
import numpy as np
import nltk
nltk.download("cmudict", quiet=True)
from nltk.corpus import cmudict

# --- CONFIG: adjust if needed ---
# manifest may be in /kaggle/working/manifests or /mnt/data if you uploaded it here
manifest_candidates = [
    Path("/kaggle/working/manifests/train-clean-100.tsv"),
    Path("/mnt/data/train-clean-100.tsv"),
    Path("manifests/train-clean-100.tsv")
]
MANIFEST = next((p for p in manifest_candidates if p.exists()), None)
if MANIFEST is None:
    raise FileNotFoundError("Manifest not found. Expected at one of: " + ", ".join(str(p) for p in manifest_candidates))
print("Using manifest:", MANIFEST)

# small utilities: CMUdict phonemizer + minimal phoneme->features (covers common ARPAbet)
cmu = cmudict.dict()
def word_to_phonemes(word):
    w = word.lower()
    if w in cmu:
        phones = cmu[w][0]
        return [p.rstrip("012") for p in phones]
    return []

def sentence_to_phonemes(sentence):
    words = [w.strip(".,!?;:()[]'\"").lower() for w in sentence.split()]
    phs = []
    for w in words:
        phs += word_to_phonemes(w)
    return phs

FEATURE_LIST = [
 'consonant','sonorant','fricative','nasal','stop','approximant','affricate','liquid','vowel','semivowel','continuant',
 'alveolar','palatal','dental','glottal','labial','velar','mid','high','low','front','back','central','anterior','posterior','retroflex','bilabial','coronal','dorsal',
 'long','short','monophthong','diphthong','round','voiced'
]
N_FEATURES = len(FEATURE_LIST)
from collections import defaultdict
PHONEME_TO_FEATURES = defaultdict(set)
def add(ph, *feats):
    PHONEME_TO_FEATURES[ph].update(feats)

# minimal mapping (covers standard ARPAbet used by CMUdict)
# vowels
add('AA','vowel','low','back','monophthong'); add('AE','vowel','low','front','monophthong')
add('AH','vowel','mid','central','short','monophthong'); add('AO','vowel','low','back','monophthong')
add('AW','vowel','diphthong','back'); add('AY','vowel','diphthong','front')
add('EH','vowel','mid','front','monophthong'); add('ER','vowel','mid','central','monophthong','round')
add('EY','vowel','diphthong','front'); add('IH','vowel','high','front','short')
add('IY','vowel','high','front','long','monophthong'); add('OW','vowel','diphthong','back')
add('OY','vowel','diphthong','back'); add('UH','vowel','high','back','short'); add('UW','vowel','high','back','long','round')
# stops
add('P','consonant','stop','bilabial'); add('B','consonant','stop','bilabial','voiced')
add('T','consonant','stop','alveolar'); add('D','consonant','stop','alveolar','voiced')
add('K','consonant','stop','velar'); add('G','consonant','stop','velar','voiced')
# fricatives
add('F','consonant','fricative','labial'); add('V','consonant','fricative','labial','voiced')
add('TH','consonant','fricative','dental'); add('DH','consonant','fricative','dental','voiced')
add('S','consonant','fricative','alveolar'); add('Z','consonant','fricative','alveolar','voiced')
add('SH','consonant','fricative','palatal'); add('ZH','consonant','fricative','palatal','voiced'); add('HH','consonant','fricative','glottal')
# affricates / nasals / approximants
add('CH','consonant','affricate','palatal'); add('JH','consonant','affricate','palatal','voiced')
add('M','consonant','nasal','bilabial','voiced'); add('N','consonant','nasal','alveolar','voiced'); add('NG','consonant','nasal','velar','voiced')
add('L','consonant','liquid','alveolar','voiced'); add('R','consonant','liquid','coronal','voiced')
add('W','consonant','semivowel','labial','voiced'); add('Y','consonant','semivowel','palatal','voiced')

def phonemes_to_feature_sequences(phonemes):
    """Return list of N_FEATURES lists of global token ids: +att=2*i, -att=2*i+1"""
    seqs = [[] for _ in range(N_FEATURES)]
    for ph in phonemes:
        feats = PHONEME_TO_FEATURES.get(ph, set())
        for i, feat in enumerate(FEATURE_LIST):
            if feat in feats:
                seqs[i].append(2*i)
            else:
                seqs[i].append(2*i+1)
    return seqs

# --- Dataset (reads manifest, returns waveform numpy and phonemes) ---
class SmallLibriDataset(torch.utils.data.Dataset):
    def __init__(self, manifest_tsv, max_items=None):
        self.rows = []
        with open(manifest_tsv, "r", encoding="utf-8") as f:
            rdr = csv.DictReader(f, delimiter="\t")
            for i,row in enumerate(rdr):
                self.rows.append((row['audio_path'], row['transcript']))
                if max_items and i+1>=max_items:
                    break
    def __len__(self):
        return len(self.rows)
    def __getitem__(self, idx):
        path, transcript = self.rows[idx]
        waveform, sr = torchaudio.load(path)
        if sr != fe.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, fe.sampling_rate)
        if waveform.dim()>1:
            waveform = waveform.mean(dim=0)
        # return numpy 1D float32 (feature extractor accepts list of arrays)
        wav_np = waveform.numpy().astype(np.float32)
        # phonemes
        phs = sentence_to_phonemes(transcript)
        feat_seqs = phonemes_to_feature_sequences(phs)
        return wav_np, phs, feat_seqs, path

# collate: use feature-extractor `fe` to get input_values & attention_mask (pt tensors)
def collate_batch(batch):
    wavs = [item[0] for item in batch]  # numpy arrays
    phs = [item[1] for item in batch]
    feat_seqs = [item[2] for item in batch]
    paths = [item[3] for item in batch]
    inputs = fe(wavs, sampling_rate=fe.sampling_rate, return_tensors="pt", padding=True)
    return inputs, feat_seqs, phs, paths

# --- create small dataloader (2 examples) ---
ds = SmallLibriDataset(MANIFEST, max_items=200)   # limit to 200 reading time
print("Dataset size (using max_items=200):", len(ds))
dl = torch.utils.data.DataLoader(ds, batch_size=2, shuffle=True, collate_fn=collate_batch)

# --- build linear head and device setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
# freeze feature extractor of wav2vec if not already
for p in model.feature_extractor.parameters():
    p.requires_grad = False
hidden = model.config.hidden_size
out_dim = 2 * N_FEATURES + 1
linear_head = nn.Linear(hidden, out_dim).to(device)
model.to(device).eval()

# take one batch and run forward
batch_inputs, batch_feat_seqs, batch_phs, batch_paths = next(iter(dl))
input_values = batch_inputs["input_values"].to(device)   # shape (B, L)
attention_mask = batch_inputs.get("attention_mask", None)
if attention_mask is not None:
    attention_mask = attention_mask.to(device)

with torch.no_grad():
    wav2_out = model(input_values, attention_mask=attention_mask)
    hs = wav2_out.last_hidden_state     # (B, T, H)
    logits = linear_head(hs)            # (B, T, C)

print("Batch audio paths:", batch_paths)
print("Example phonemes (sample 0):", batch_phs[0][:60])
print("Example feature-seq lengths for sample 0 (per-feature):", [len(s) for s in batch_feat_seqs[0][:8]], " ...")
print("Hidden states shape (B, T, H):", hs.shape)
print("Logits shape (B, T, C):", logits.shape, " expected C:", out_dim)


Using manifest: /kaggle/working/manifests/train-clean-100.tsv
Dataset size (using max_items=200): 200
Device: cpu
Batch audio paths: ['/kaggle/input/librispeech-100/LibriSpeech/train-clean-100/103/1240/103-1240-0020.flac', '/kaggle/input/librispeech-100/LibriSpeech/train-clean-100/103/1240/103-1240-0054.flac']
Example phonemes (sample 0): ['B', 'AO', 'R', 'D', 'ER', 'D', 'W', 'IH', 'DH', 'W', 'AY', 'L', 'D', 'R', 'OW', 'Z', 'B', 'UH', 'SH', 'AH', 'Z', 'IH', 'T', 'S', 'N', 'OW', 'W', 'AH', 'N', 'D', 'ER', 'M', 'AE', 'TH', 'Y', 'UW', 'AH', 'N', 'D', 'M', 'AA', 'R', 'IH', 'L', 'AH', 'AA', 'R', 'B', 'OW', 'TH', 'AH', 'L', 'IH', 'T', 'AH', 'L', 'AA', 'D', 'L', 'IH']
Example feature-seq lengths for sample 0 (per-feature): [153, 153, 153, 153, 153, 153, 153, 153]  ...
Hidden states shape (B, T, H): torch.Size([2, 816, 1024])
Logits shape (B, T, C): torch.Size([2, 816, 71])  expected C: 71
